In [124]:
from sklearn.model_selection import train_test_split

#Weak Learners
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import roc_auc_score

from sklearn.utils import resample

import pandas as pd
import numpy as np

In [102]:
dataset = pd.read_csv("iris_dataset.csv")
df.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [125]:
labelEncoder = LabelEncoder()
labelEncoder.fit(["Iris-setosa", "Iris-virginica", "Iris-versicolor"])
df["Class"] = labelEncoder.transform(dataset['Class'].values)

In [69]:
df.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


## Estandarización de Variable

In [132]:
df_scaled = df.copy()
cols_to_scale = ["PetalLength", "PetalLength", "PetalLength", "PetalWidth"]

features = df_scaled[cols_to_scale]

scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)

df_scaled[cols_to_scale] = features
df_scaled.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Class
0,5.1,3.5,-1.341272,-1.312977,0
1,4.9,3.0,-1.341272,-1.312977,0
2,4.7,3.2,-1.398138,-1.312977,0
3,4.6,3.1,-1.284407,-1.312977,0
4,5.0,3.6,-1.341272,-1.312977,0


## Seleccionamos Variables Target 

In [144]:
X = df_scaled.drop('Class', axis=1)
y = df_scaled.Class

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2021, shuffle=True)

## Configuración y Lista de Modelos

In [145]:
lr = 'LogisticRegression().fit(X_train, y_train)'
svm = 'SVC(gamma="scale", kernel="linear").fit(X_train, y_train)'
dt = 'DecisionTreeClassifier(criterion="gini", max_depth=4).fit(X_train, y_train)'
knn = 'KNeighborsClassifier(n_neighbors=7).fit(X_train, y_train)'
nb = 'GaussianNB().fit(X_train, y_train)'
lda = 'LinearDiscriminantAnalysis(solver="svd", store_covariance=True).fit(X_train, y_train)'
qda = 'QuadraticDiscriminantAnalysis(store_covariance=True).fit(X_train, y_train)'
rf = 'RandomForestClassifier(n_estimators=50).fit(X_train, y_train)'

model_list = {'lr': lr, 
              'svm': svm,
              'dt': dt,
              'knn': knn,
              'nb': nb,
              'lda': lda,
              'qda': qda,
              'rf': rf}

model_list

{'lr': 'LogisticRegression().fit(X_train, y_train)',
 'svm': 'SVC(gamma="scale", kernel="linear").fit(X_train, y_train)',
 'dt': 'DecisionTreeClassifier(criterion="gini", max_depth=4).fit(X_train, y_train)',
 'knn': 'KNeighborsClassifier(n_neighbors=7).fit(X_train, y_train)',
 'nb': 'GaussianNB().fit(X_train, y_train)',
 'lda': 'LinearDiscriminantAnalysis(solver="svd", store_covariance=True).fit(X_train, y_train)',
 'qda': 'QuadraticDiscriminantAnalysis(store_covariance=True).fit(X_train, y_train)',
 'rf': 'RandomForestClassifier(n_estimators=50).fit(X_train, y_train)'}

## Creamos procedimiento de Baggin

In [146]:
trainSet = pd.concat([X_train, y_train], axis=1)
trainSet.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Class
83,6.3,2.9,1.047087,0.790591,2
149,5.9,3.0,0.762759,0.790591,2
136,5.7,2.9,0.250967,0.133226,1
59,5.6,2.5,0.080370,-0.129720,1
88,6.7,2.5,1.160819,0.790591,2


In [154]:
train_models_list = {}

for model_name, model in model_list.items():
    sample = resample(trainSet, replace=True, n_samples=200)
    X_train = sample.drop('Class', axis=1)
    y_train = sample.Class
    train_model = eval(model)
    train_models_list[model_name] = train_model

train_models_list

{'lr': LogisticRegression(),
 'svm': SVC(kernel='linear'),
 'dt': DecisionTreeClassifier(max_depth=4),
 'knn': KNeighborsClassifier(n_neighbors=7),
 'nb': GaussianNB(),
 'lda': LinearDiscriminantAnalysis(store_covariance=True),
 'qda': QuadraticDiscriminantAnalysis(store_covariance=True),
 'rf': RandomForestClassifier(n_estimators=50)}

## Predicciones

In [158]:
acc_scores = {}
auc_roc_scores = {}

result_matrix = pd.DataFrame(columns=np.arange(0, len(y_test), 1))

for model_name, train_model in train_models_list.items():
    predicciones = train_model.predict(X_test)
    acc = accuracy_score(y_test, predicciones)
    aucroc = roc_auc_score(y_test, predicciones)
    acc_scores[model_name] = acc
    auc_roc_scores[model_name] = aucroc
    tempDf = pd.DataFrame(predicciones).T
    result_matrix = result_matrix.append(tempDf)

result_matrix.index = list(train_models_list.keys())

ValueError: multi_class must be in ('ovo', 'ovr')

acc_scores

In [156]:
result_matrix

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44


## Votación

In [157]:
votacion = result_matrix.apply(pd.value_counts)
votacion

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44


In [117]:
final_predict = []

for columnName, columnData in votacion.iteritems():
    column_result = columnData.values
    final_predict.append(np.nanargmax(column_result, axis=0))
    
final_predict

ValueError: attempt to get argmax of an empty sequence

In [118]:
accEnsamble = accuracy_score(y_test, final_predict)
aucrocEnsamble = roc_auc_score(y_test, final_predict)
aucrocEnsamble

ValueError: Found input variables with inconsistent numbers of samples: [45, 0]

In [119]:
auc_roc_scores

{}